#PyTorch - MNIST Addition of two digits


Write a neural network that can:

take 2 inputs:
an image from the MNIST dataset (say 5), and
a random number between 0 and 9, (say 7)
and gives two outputs:
the "number" that was represented by the MNIST image (predict 5), and
the "sum" of this number with the random number and the input image to the network (predict 5 + 7 = 12)



you can mix fully connected layers and convolution layers
you can use one-hot encoding to represent the random number input as well as the "summed" output.

Random number (7) can be represented as 0 0 0 0 0 0 0 1 0 0
Sum (13) can be represented as:
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
0b1101 (remember that 4 digits in binary can at max represent 15, so we may need to go for 5 digits. i.e. 10010


In [ ]:
# Pytorch's tensors are similar to Numpy's ndarrays
!pip install torch

#Import libs

In [ ]:
import torch
import torchvision #datasets, models, transforms utils
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchsummary import summary
import random as random
import torch.nn.functional as F
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# #Evaluate the model in GPU
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# torch.cuda.set_device(device)

cpu


#Prepare data set for first 0 - 9 input

In [ ]:
class MyDataset(Dataset):
  def __init__(self):
    self.digit_data = (1,2,3,4,5,6,7,8,9,0)
    self.image_train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))

  def __getitem__(self,index):
    return self.digit_data[index]

  def __len__(self):
    return len(self.digit_data)
  
  #Random numberf generation
  def getRandomNumber(self,batchsize):
    return random.sample(range(0, 10), batchsize)

myData = MyDataset()

In [ ]:
#compare the actual label with predicted label
def get_num_correct(preds, labels):
  # return preds.argmax(dim=1).eq(labels).sum().item()
  return preds.argmax(dim=1).eq(labels.argmax(dim=1)).sum().item()

#NN Architecture 

In [ ]:

class Network1(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    self.fc1 = nn.Linear(in_features=(12 * 4 * 4), out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out_1 = nn.Linear(in_features=60, out_features=10)

    self.fc3 = nn.Linear(in_features=20,out_features=60)
    self.fc4 = nn.Linear(in_features=60,out_features=60)
    self.out_2 = nn.Linear(in_features=60, out_features=29)

  
  def forward(self, input_image, input_digit):
    # print("input size ---> ",input_image.shape,input_digit.shape)
    # input layer
    x = input_image
    d = input_digit

    # conv1 layer
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

    # conv2 layer
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

    # reshapre
    x = x.reshape(-1, (12 * 4 * 4)) # 1 * 192
    # print("d shpae before reshape",d.shape)
    d = d.reshape(-1,10) 

    # fc1 layer
    x = self.fc1(x) # 192 | 120
    x = F.relu(x)

    # fc2 layer
    x = self.fc2(x)  #  120 | 60
    x = F.relu(x)

    # output layer
    x = self.out_1(x)   # 60 | 10
    # x = F.softmax(x)
    # print("shape of x and d",x.shape,d.shape) 

    x = torch.cat((x, d), dim=1) #  10 +19 | 29

    # print("shape of x after cat",x.shape) 

    # fc3 layer
    x = self.fc3(x)    # 11 | 20
    x = F.relu(x)

    #fc4 layer
    x = self.fc4(x)    #  20 | 10
    x = F.relu(x)

    # output layer
    x = self.out_2(x)  #  20 | 2
    # x = F.relu(x)
    x = F.softmax(x,dim=1)
    return x

In [ ]:
network1 = Network1()
for name, param in network.named_parameters():
  print(name, '\t\t', param.shape)

batch_size = 10
train_loader = torch.utils.data.DataLoader(myData.image_train_set,batch_size=batch_size)
# train_loader_digits = torch.utils.data.DataLoader(myData,batch_size=batch_size)
optimizer = torch.optim.Adam(network.parameters(),lr=0.001)

# summary(network,[(1,28,28),([1,10])])
# print(summary(network,[(1,28,28)]))

count = 10
for epoch in range(10):
  
  total_loss = 0
  total_correct = 0

  for batch in train_loader:
    images, labels = batch
    randomNumTensor = torch.as_tensor(myData.getRandomNumber(batch_size))
    # print(randomNumTensor.shape,labels.shape)
    # print(randomNumTensor,labels)
    # print(addition)
    finalLabel = torch.cat((F.one_hot(labels, num_classes=10),F.one_hot(randomNumTensor + labels, num_classes=19)),1).type(torch.float)

    preds = network1(images,torch.as_tensor(F.one_hot(randomNumTensor, num_classes=10), dtype=torch.float))
    # print("label",labels,torch.as_tensor(F.one_hot(randomNumTensor, num_classes=10), dtype=torch.float))
    # print("Pred",preds)
    # print(preds[:,0:10].argmax(dim=1))
    # print(preds[:,10:].argmax(dim=1))
    loss_digit = F.mse_loss(preds,finalLabel)

    optimizer.zero_grad()
    loss_digit.backward()
    optimizer.step()

    total_loss =  loss_digit.item()
    # print("finalLabel,preds",finalLabel[:10],preds[:10])
    total_correct += get_num_correct(preds,finalLabel)
  print(
      "epoch", epoch, 
      "total_correct:", total_correct, 
      "loss:", total_loss
  )

conv1.weight 		 torch.Size([6, 1, 5, 5])
conv1.bias 		 torch.Size([6])
conv2.weight 		 torch.Size([12, 6, 5, 5])
conv2.bias 		 torch.Size([12])
fc1.weight 		 torch.Size([120, 192])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([60, 120])
fc2.bias 		 torch.Size([60])
out_1.weight 		 torch.Size([10, 60])
out_1.bias 		 torch.Size([10])
fc3.weight 		 torch.Size([60, 20])
fc3.bias 		 torch.Size([60])
fc4.weight 		 torch.Size([60, 0])
fc4.bias 		 torch.Size([60])
out_2.weight 		 torch.Size([29, 60])
out_2.bias 		 torch.Size([29])
epoch 0 total_correct: 1825 loss: 0.06541407108306885
epoch 1 total_correct: 1857 loss: 0.06542697548866272
epoch 2 total_correct: 1813 loss: 0.06547296047210693
epoch 3 total_correct: 1804 loss: 0.06545644253492355
epoch 4 total_correct: 1792 loss: 0.06534674763679504
epoch 5 total_correct: 1807 loss: 0.06550444662570953
epoch 6 total_correct: 1790 loss: 0.06547430902719498


In [ ]:

# class Network(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5) 
#     self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
#     self.fc1 = nn.Linear(in_features=(12 * 4 * 4), out_features=120)
#     self.fc2 = nn.Linear(in_features=120, out_features=60)
#     self.out_1 = nn.Linear(in_features=60, out_features=10)

#     self.fc3 = nn.Linear(in_features=11,out_features=20)
#     self.fc4 = nn.Linear(in_features=20,out_features=20)
#     self.out_2 = nn.Linear(in_features=20, out_features=2)

  
#   def forward(self, input_image, input_digit):
#     print("input size ---> ",input_image.shape,input_digit.shape)
#     # input layer
#     x = input_image
#     d = input_digit

#     # conv1 layer
#     x = self.conv1(x)
#     x = F.relu(x)
#     x = F.max_pool2d(x, kernel_size=2, stride=2) # 28 | 24 | 12

#     # conv2 layer
#     x = self.conv2(x)
#     x = F.relu(x)
#     x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 8 | 4 >> 12x4x4

#     # reshapre
#     x = x.reshape(-1, (12 * 4 * 4)) # 1 * 192
#     # print("d shpae before reshape",d.shape)
#     d = d.reshape(-1,1) 

#     # fc1 layer
#     x = self.fc1(x) # 192 | 120
#     x = F.relu(x)

#     # fc2 layer
#     x = self.fc2(x)  #  120 | 60
#     x = F.relu(x)

#     # output layer
#     x = self.out_1(x)   # 60 | 10
#     # x = F.softmax(x)
#     # print("shape of x and d",x.shape,d.shape) 

#     x = torch.cat((x, d), dim=1) #  10 +1 | 11

#     # print("shape of x after cat",x.shape) 

#     # fc3 layer
#     x = self.fc3(x)    # 11 | 20
#     x = F.relu(x)

#     #fc4 layer
#     x = self.fc4(x)    #  20 | 10
#     x = F.relu(x)

#     # output layer
#     x = self.out_2(x)  #  20 | 2
#     x = F.relu(x)
#     return x

In [ ]:
# network = Network()
# for name, param in network.named_parameters():
#   print(name, '\t\t', param.shape)

# batch_size = 10
# train_loader = torch.utils.data.DataLoader(myData.image_train_set,batch_size=batch_size)
# # train_loader_digits = torch.utils.data.DataLoader(myData,batch_size=batch_size)
# optimizer = torch.optim.Adam(network.parameters(),lr=0.001)

# # summary(network,[(1,28,28),([1,10])])
# # print(summary(network,[(1,28,28)]))

In [ ]:

# count = 10
# for epoch in range(1):
  
#   total_loss = 0
#   total_correct = 0

#   for batch in train_loader:
#     images, labels = batch
#     randomNumTensor = torch.as_tensor(myData.getRandomNumber(batch_size)).type(torch.float)
#     # print(randomNumTensor.shape,labels.shape)
#     addition = randomNumTensor+labels
#     # print(randomNumTensor,labels)
#     # print(addition)
#     Sumlabels = torch.cat((labels,addition),0)
#     # print("Sumlabels before reshape",Sumlabels)
#     Sumlabels = Sumlabels.reshape(10,2)
#     # print("Sumlabels after reshape",Sumlabels)
#     preds = network(images,randomNumTensor)
#     # print("Sumlabels.shape,preds.shape",Sumlabels.shape,preds.shape)
#     print("label",labels,randomNumTensor,addition)
#     print("Pred",preds)
#     loss_digit = F.mse_loss(preds,Sumlabels)

#     optimizer.zero_grad()
#     loss_digit.backward()
#     optimizer.step()

#     total_loss =  loss_digit.item()
#     total_correct += get_num_correct(preds,labels)
#   print(
#       "epoch", epoch, 
#       "total_correct:", total_correct, 
#       "loss:", total_loss
#   )